<a href="https://colab.research.google.com/github/jisoolee11/Deep-Learning/blob/main/Transcription/2020_dacon_cup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>2020 DACON CUP baseline 코드 입니다.

In [1]:
cd "/content/drive/My Drive/Data"

/content/drive/My Drive/Data


In [2]:
# !unzip -qq "/content/drive/My Drive/Data/2020_dacon_cup"

In [3]:
# libraray import
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')  #경고 메세지 무시하고 숨기기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
torch.manual_seed(1015)  #gpu연산 무작위 고정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# 데이터 불러오기
train = pd.read_csv("/content/drive/My Drive/Data/open_data/1차_open_data/train.csv", encoding='euc-kr')  #한글을 2byte로 인코딩(다른 언어x)
train['DateTime'] = pd.to_datetime(train.DateTime)
train['date'] = train.DateTime.dt.date
train = train.groupby('date').sum().reset_index()  # 날짜별 그룹별 집계
train.head()


,date,사용자,세션,신규방문자,페이지뷰
0,2018-09-09,281,266,73,1826
1,2018-09-10,264,247,51,2092
2,2018-09-11,329,310,58,1998
3,2018-09-12,300,287,45,2595
4,2018-09-13,378,344,50,3845


In [5]:
# 데이터 수정
# scaling
mini = train.iloc[:, 1:].min()
size = train.iloc[:, 1:].max() - train.iloc[:, 1:].min()
train.iloc[:, 1:] = (train.iloc[:, 1:] - mini) / size
train.head()

input_window = 30
output_window = 7

window_x = np.zeros((train.shape[0] - (input_window + output_window), input_window, 4))
window_y = np.zeros((train.shape[0] - (input_window + output_window), output_window, 4))

for start in range(train.shape[0] - (input_window +output_window)):
    end = start +input_window
    window_x[start, :, :] = train.iloc[start: end, 1:].values
    window_y[start, :, :] = train.iloc[end: end + output_window, 1:].values

In [6]:
# model 생성
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(LSTM, self).__init__()
    self.hidden_size = hidden_size
    # batch_first: RNN이나 LSTM 같은 경우는 batch_size가 아닌 sequence가 가장 먼저 나온다
    self.lstm = nn.LSTM(input_size=input_size,
                        hidden_size=hidden_size,
                        batch_first=True)
    self.hidden_lstm = nn.LSTM(input_size=hidden_size,
                               hidden_size=hidden_size,
                               batch_first=True)
  
    self.time_fc = nn.Linear(hidden_size, 4)  #선형회귀 
  
  def forward(self, x_time):
    out_time, _ = self.lstm(x_time)
    out_time, _ = self.hidden_lstm(out_time)
    out_time, _ = self.hidden_lstm(out_time)
    out_time, _ = self.hidden_lstm(out_time)


    out_time = self.time_fc(out_time[:,-7:, :])

    return out_time.view(-1, 7, 4)  #차원-1, 7행 4열??

model = LSTM(input_size = 4, hidden_size = 30).to(device)    

In [7]:
# 학습
window_x = torch.tensor(window_x).float().to(device)
window_y = torch.tensor(window_y).float().to(device)

# train model
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-2)
criterion = nn.MSELoss(size_average= True)  #data size에 영향x
num_epochs = 500
train_error = []
for t in range(num_epochs):
  train_pred = model(window_x)
  loss = criterion(train_pred, window_y)
  train_error.append(loss)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  if t % 10 == 0 and t != 0:
    print(f"{t} Epochs train MSE: {loss.item():1.5f}")

10 Epochs train MSE: 0.02835
20 Epochs train MSE: 0.01247
30 Epochs train MSE: 0.01194
40 Epochs train MSE: 0.00882
50 Epochs train MSE: 0.00821
60 Epochs train MSE: 0.00813
70 Epochs train MSE: 0.00802
80 Epochs train MSE: 0.00795
90 Epochs train MSE: 0.00786
100 Epochs train MSE: 0.00776
110 Epochs train MSE: 0.00755
120 Epochs train MSE: 0.00709
130 Epochs train MSE: 0.00646
140 Epochs train MSE: 0.00647
150 Epochs train MSE: 0.00667
160 Epochs train MSE: 0.00626
170 Epochs train MSE: 0.00591
180 Epochs train MSE: 0.00571
190 Epochs train MSE: 0.00555
200 Epochs train MSE: 0.00548
210 Epochs train MSE: 0.00541
220 Epochs train MSE: 0.00530
230 Epochs train MSE: 0.00624
240 Epochs train MSE: 0.00527
250 Epochs train MSE: 0.00507
260 Epochs train MSE: 0.00486
270 Epochs train MSE: 0.00913
280 Epochs train MSE: 0.00610
290 Epochs train MSE: 0.00538
300 Epochs train MSE: 0.00517
310 Epochs train MSE: 0.00496
320 Epochs train MSE: 0.00482
330 Epochs train MSE: 0.00469
340 Epochs train MS

In [11]:
# 예측
submission = pd.read_csv("/content/drive/My Drive/Data/open_data/1차_open_data/submission.csv", encoding = 'euc-kr')

last_month = train.iloc[-30:, 1:].values[np.newaxis, ...]
last_month = torch.tensor(last_month).float().to(device)

for start in range((len(submission) - output_window)//7 + 2):
    start = start * 7
    next_week = model(last_month)
    last_month = torch.cat([last_month[:, 7:, :], next_week], axis = 1)

    pred_week = next_week.cpu().detach().numpy().reshape(output_window, 4)
    pred_week = pred_week *size.values + mini.values
    pred_week = pred_week.astype(int)

    if start/7 == (len(submission) - output_window)//7 + 1:
        submission.iloc[start:, 1:] = pred_week[-submission.iloc[start:, 1:].shape[0]:, :]
    else:
        submission.iloc[start: start + output_window, 1:] = pred_week
submission


,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,3472,3424,891,91050
1,2020-11-10,3300,3296,886,85737
2,2020-11-11,2955,3009,789,75937
3,2020-11-12,2276,2331,576,57679
4,2020-11-13,1793,1808,411,43594
...,...,...,...,...,...
56,2021-01-04,3005,2909,711,81326
57,2021-01-05,2467,2421,580,66552
58,2021-01-06,1839,1807,414,47243
59,2021-01-07,1798,1768,411,46659


In [12]:
# 예측파일 저장
submission.to_csv('submission.csv', index = False, encoding = 'euc-kr')